In [ ]:
# Dem Libraries!
%matplotlib inline
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from keras.models import Model
from keras.layers import Input, Reshape
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling1D, Conv1D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD
from keras.callbacks import TensorBoard
import pretty_midi

#Paths
training_path = './Training Stage 1/'

In [ ]:
# To get the sine waves data
# n_samples : Number of sine waves you want generated
def sample_data(training_path):
    training_path = './Training Stage 1/'
    filenames = []
    lens = []
    fin_midi_stream = []
    for filename in os.listdir(training_path):
        filenames.append(os.path.join(training_path,filename))
    fin_midi_streama = []
    fin_midi_files = []
    for file in filenames:
        temp = pretty_midi.PrettyMIDI(file)
        melody = temp.instruments[0]
    #     chord = temp.instruments[1]
        melody_math = melody.get_piano_roll(16)
        curr_stream = np.argmax(melody_math, axis=0)
        fin_midi_streama.extend(curr_stream)
    chop_size = 256
    num_of_chops = len(fin_midi_streama)//chop_size
    for i in range(num_of_chops):
        fin_midi_files.append(fin_midi_streama[(i)*chop_size:(i+1)*chop_size])
    
    return fin_midi_files

ALL_DATA = sample_data(training_path)

def get_ALL_data(n_samples):
    return ALL_DATA[:n_samples]

In [ ]:
#Generator Model :
# Needs to be able to take in random noise and generate realistic looking output data

def get_generative(G_in, dense_dim=400, out_dim=256, lr=1e-3):
    x = Dense(dense_dim)(G_in)
    x = Activation('tanh')(x)
    G_out = Dense(out_dim, activation='tanh')(x)
    G = Model(G_in, G_out)
    opt = SGD(lr=lr)
    G.compile(loss='binary_crossentropy', optimizer=opt)
    return G, G_out

G_in = Input(shape=[16])
G, G_out = get_generative(G_in)
G.summary()

In [ ]:
#Discriminator Model :
# Needs to be able to take in a generated signal and distinguish between real and fake(generated)

def get_discriminative(D_in, lr=1e-3, drate=.25, n_channels=50, conv_sz=5, leak=.2):
    x = Reshape((-1, 1))(D_in)
    
    x = Conv1D(n_channels, conv_sz, activation='relu')(x)
    x = Dropout(drate)(x)
    x = Flatten()(x)
    x = Dense(n_channels)(x)
    D_out = Dense(2, activation='sigmoid')(x)
    D = Model(D_in, D_out)
    dopt = Adam(lr=lr)
    D.compile(loss='binary_crossentropy', optimizer=dopt)
    return D, D_out

D_in = Input(shape=[256])
D, D_out = get_discriminative(D_in)
D.summary()

In [ ]:
# Function to make sure the Discriminators weights are frozen, while training the Generator
def set_trainability(model, trainable=False):
    model.trainable = trainable
    for layer in model.layers:
        layer.trainable = trainable

        
def make_gan(GAN_in, G, D):
    set_trainability(D, False)
    x = G(GAN_in)
    GAN_out = D(x)
    GAN = Model(GAN_in, GAN_out)
    GAN.compile(loss='binary_crossentropy', optimizer=G.optimizer)
    return GAN, GAN_out

GAN_in = Input([16])
GAN, GAN_out = make_gan(GAN_in, G, D)
GAN.summary()

In [ ]:
# Function to get a concatenated input of real_data and fake_data with appropriate concatenated labels
def sample_data_and_gen(G, noise_dim=16, n_samples=15000):
    # Generated Real Samples 
    XT = get_ALL_data(n_samples)
    # Get the random noise that'll be used to generate the fake data
    XN_noise = np.random.uniform(73, 87, size=[n_samples, noise_dim])
    # Generate the Fake Samples
    XN = G.predict(XN_noise)
    # Concatenat the Real and Fake Data
    X = np.concatenate((XT, XN))
    # Concatenate the labels and set the appropriate values
    y = np.zeros((2*n_samples, 2))
    y[:n_samples, 1] = 1
    y[n_samples:, 0] = 1
    return X, y

def pretrain(G, D, noise_dim=16, n_samples=15000, batch_size=32):
    # Get the real samples for the pretraining of the discriminator
    X, y = sample_data_and_gen(G, n_samples=n_samples, noise_dim=noise_dim)
    # Allow the Discriminator to be trained
    set_trainability(D, True)
    # Fit the Discriminator with 1 epoch
    D.fit(X, y, epochs=1, batch_size=batch_size)


In [ ]:
# Perform One Train run for the Determiner
pretrain(G, D)

In [ ]:
# Get sample noise for the GAN training cycle
def sample_noise(G, noise_dim=16, n_samples=15000):
    X = np.random.uniform(73, 87, size=[n_samples, noise_dim])
    y = np.zeros((n_samples, 2))
    y[:, 1] = 1
    return X, y

# Training the GAN by backpropagating the losses of the discriminator on the generated samples, 
# whilst freezing discriminator weights

def train(GAN, G, D, epochs=50, n_samples=15000, noise_dim=16, batch_size=32, verbose=False, v_freq=5):
    d_loss = []
    g_loss = []
    e_range = range(epochs)
    if verbose:
        e_range = tqdm(e_range)
    #For each EPOCH
    for epoch in e_range:
        # Get True and Generated Samples with Labels
        X, y = sample_data_and_gen(G, n_samples=n_samples, noise_dim=noise_dim)
        # Allow Discriminator to be Trained
        set_trainability(D, True)
        # Evaluate the Discriminator loss on this set(only to record loss)
        d_loss.append(D.train_on_batch(X, y))
        
        # Now just get the generated samples
        X, y = sample_noise(G, n_samples=n_samples, noise_dim=noise_dim)
        # Freeze the Discriminator Weights
        set_trainability(D, False)
        # Evaluate the Generator loss on this set
        g_loss.append(GAN.train_on_batch(X, y))
        if verbose and (epoch + 1) % v_freq == 0:
            print("Epoch #{}: Generative Loss: {}, Discriminative Loss: {}".format(epoch + 1, g_loss[-1], d_loss[-1]))
    return d_loss, g_loss


In [ ]:
d_loss, g_loss = train(GAN, G, D, verbose=True)

In [ ]:
# This step is not applicable here for now. 

# N_VIEWED_SAMPLES = 2
# data_and_gen, _ = sample_data_and_gen(G, n_samples=N_VIEWED_SAMPLES)

# #To View the Generated Output
# pd.DataFrame(np.transpose(data_and_gen[N_VIEWED_SAMPLES:])).plot()
# #To View a smoothened version of the Generated Output(using a rollling mean)
# #pd.DataFrame(np.transpose(data_and_gen[N_VIEWED_SAMPLES:])).rolling(5).mean()[5:].plot()

In [ ]:
#Shreyans is a fucking asshole. Dipshit lil bitch. 
def fuck_shrey(d):
    d = np.round(d).astype(int)
    j = np.zeros((128,np.shape(d)[1]))
    i=0
    while (i<np.shape(d)[1]):
        j[d[0,i],i] = 1
        i = i+1
    return j

def pianoroll_to_midi(piano_roll, fs=16, program=0):
    '''Converts a Piano Roll array to a PrettyMidi object
     with a single instrument.
    
    Input:
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    
    Returns:
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    period=1./fs
    
    notes, frames = piano_roll.shape #get number of frames in our piano roll
    pm = pretty_midi.PrettyMIDI() #create a Pretty Midi object
    instrument = pretty_midi.Instrument(program=0) #specify our instrument

    #record previous pitch/velocity so we can concatenate notes together
    prev_pitch=0
    prev_velocity=0
    notelength=0
    starttime=0
    endtime=period
    
    i=0
    while i<frames: #range over the frames of the piano roll

        #need to specify velocity (100 for note, 0 for rest),  start time, end time, and pitch
        
        
        #for ith column of piano roll get the (possible) non-zero index which
        #corresponds to the pitch
        col=piano_roll[:,i] 
        colnext=piano_roll[:,i+1]
        tmp=np.nonzero(col)
        tmpnext=np.nonzero(colnext)
        
        if tmp[0].size==0: #current is rest note, don't need to worry about length
            
            velocity=0
            starttime=i*period
            endtime=period+i*period
            current_pitch=0
           
        
        else:
            #get current pitch and set the start time
            velocity=100
            current_pitch=tmp[0][0]
            starttime=i*period
            
            
            #loop over future notes to find when pitch changes
            pitchchange=False
            while pitchchange==False:
                
                #if end of song quit
                if i==frames-1:
                    endtime=period+i*period
                    break
                    
                #get next note
                colnext=piano_roll[:,i+1]
                tmpnext=np.nonzero(colnext)
                
                #if next note is a rest
                if tmpnext[0].size==0: 
                    endtime=period+i*period
                    break
                    
                #if next frame has different pitch
                elif tmpnext[0][0]!=current_pitch: 
                    endtime=period+i*period
                    pitchchange=True
            
                else:
                    #increment to next frame
                    i=i+1
                    
            
        pm_note=pretty_midi.Note(velocity=100, pitch=current_pitch, start=starttime, end=endtime)
        i=i+1
        #we have appended note, now move to next note   
        instrument.notes.append(pm_note)
        #print starttime
        #print endtime
        
    pm.instruments.append(instrument)

    return pm

In [ ]:
#Generating the tune and writing to file
def generate_tune(gan_output):
    form = np.zeros((128,len(gan_output)))
    for i in range(len(gan_output)):
        form[int(gan_output[i])][i]=60
    form_midi = pianoroll_to_midi(form,16,0)
    form_midi.write('AI_Tune.mid')
    return form_midi

# Make Music Function
def make_music(dble_octvs):
    xyz, _ = sample_data_and_gen(G, n_samples=10)
    xyz_concat = []
    for sixteen_notes in xyz:
        xyz_concat.extend(sixteen_notes)
    generate_tune(xyz_concat)

In [ ]:
#how many double octaves(16 notes) are to be generated?
double_octaves = 10
make_music(double_octaves)